In [51]:
import numpy as np
import pandas as pd
import os

In [52]:
#Clean Users dataframe
#Importing dataframe from "dirty" folder
df = pd.read_parquet(f"dirty/users.parquet")

#Drop information about users with no adults 
drop_indexes = df[((df["count_adults"]==0) & (df["count_people"]>0)) |
   ((df["count_adults"]==0) & (df["count_pets"]>0))].index

df_clean = df.copy()
df_clean.drop(index=drop_indexes, inplace=True)


# 95% of rows with missing info about people
# Comparing df.isna() with df.notna(), similar user_segment and user_nuts1 distribution
# Fill every missing value following column distribution

missing_columns = ["count_adults", "count_children", "count_babies", "count_pets", "user_nuts1"]

for column in missing_columns:
    missing_values = df_clean[column].isna()
    s = df_clean[column].value_counts(normalize=True)
    np.random.seed(99)
    df_clean.loc[missing_values, column] = np.random.choice(s.index, size=missing_values.sum(), p=s.values)

#Recalculating count_people column where np.nan
missing_values = df_clean["count_people"].isna()
df_clean.loc[missing_values, "count_people"] = (df_clean.loc[missing_values, "count_adults"] 
                                                + df_clean.loc[missing_values, "count_children"] 
                                                + df_clean.loc[missing_values, "count_babies"])

#Exporting dataframe to clean folder
df_clean.to_parquet(f"clean/users.parquet")


In [53]:
#Merging orders dataframe

df_orders = pd.read_parquet(f"dirty/orders.parquet")
df_users = pd.read_parquet(f"clean/users.parquet")
df_inventory = pd.read_parquet(f"dirty/inventory.parquet")

df_final = df_orders.copy()


#Left join to complete users information (count adults, count children, etc.)
df_final = pd.merge(
    df_final, df_users, how="left", on=["user_id"]
)


#Explode list of items whitin orders
df_explode = df_final.explode("ordered_items")
df_explode.rename(columns={"ordered_items":"variant_id"}, inplace=True)

#Merging with inventory dataframe
df_explode = pd.merge(
    df_explode, df_inventory, how="left", on=["variant_id"]
)


#Grouping information added
total_price= df_explode.groupby("id")["price"].agg("sum")
total_compare_at_price= df_explode.groupby("id")["compare_at_price"].agg("sum")
vendors = df_explode.groupby("id")["vendor"].agg(list)
product_types = df_explode.groupby("id")["product_type"].agg(list)
tags = df_explode.groupby("id")["tags"].agg(list)

#Adding grouped information added to final dataframe
df_final = pd.merge(
    df_final, total_price, how="left", on=["id"])

df_final = pd.merge(
    df_final, total_compare_at_price, how="left", on=["id"]
)

df_final = pd.merge(
    df_final, vendors, how="left", on=["id"]
)
df_final = pd.merge(
    df_final, product_types, how="left", on=["id"]
)
df_final = pd.merge(
    df_final, tags, how="left", on=["id"]
)

#If prices are zero, add np.nan with mask
df_final["price"].mask(df_final["price"]==0, np.nan, inplace=True)
df_final["compare_at_price"].mask(df_final["compare_at_price"]==0, np.nan, inplace=True)

#Export to clean folder
df_final.to_parquet(f"clean/orders.parquet")

In [49]:
df_final.head()

,id,user_id,created_at,order_date,user_order_seq,ordered_items,user_segment,user_nuts1,first_ordered_at,customer_cohort_month,count_people,count_adults,count_children,count_babies,count_pets,price,compare_at_price,vendor,product_type,tags
0,2204073066628,62e271062eb827e411bd73941178d29b022f5f2de9d37f...,2020-04-30 14:32:19,2020-04-30,1,"[33618849693828, 33618860179588, 3361887404045...",Proposition,UKI,2020-04-30 14:32:19,2020-04-01 00:00:00,4.0,4.0,0.0,0.0,0.0,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2204707520644,bf591c887c46d5d3513142b6a855dd7ffb9cc00697f6f5...,2020-04-30 17:39:00,2020-04-30,1,"[33618835243140, 33618835964036, 3361886244058...",Proposition,UKM,2020-04-30 17:39:00,2020-04-01 00:00:00,4.0,2.0,2.0,0.0,1.0,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,2204838822020,329f08c66abb51f8c0b8a9526670da2d94c0c6eef06700...,2020-04-30 18:12:30,2020-04-30,1,"[33618891145348, 33618893570180, 3361889766618...",Top Up,UKF,2020-04-30 18:12:30,2020-04-01 00:00:00,4.0,2.0,2.0,0.0,1.0,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,2208967852164,f6451fce7b1c58d0effbe37fcb4e67b718193562766470...,2020-05-01 19:44:11,2020-05-01,1,"[33618830196868, 33618846580868, 3361891234624...",Proposition,UKI,2020-05-01 19:44:11,2020-05-01 00:00:00,4.0,3.0,1.0,0.0,1.0,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,2215889436804,68e872ff888303bff58ec56a3a986f77ddebdbe5c279e7...,2020-05-03 21:56:14,2020-05-03,1,"[33667166699652, 33667166699652, 3366717122163...",Proposition,UKI,2020-05-03 21:56:14,2020-05-01 00:00:00,2.0,2.0,0.0,0.0,0.0,8.38,10.2,"[nan, nan, nan, nan, nan, nan, listerine, list...","[nan, nan, nan, nan, nan, nan, dental, dental,...","[nan, nan, nan, nan, nan, nan, [mouthwash], [m..."
